# 0. Import required packages

In [2]:
import os
import yaml
from massibo_ana.preprocess.DataPreprocessor import DataPreprocessor

# 1. Load the input parameters and change the WD to the workspace

In [3]:
# Write here the ABSOLUTE path to the input parameters file
input_parameters_filepath = ''

with open(
        input_parameters_filepath,
        "r",
        encoding="utf-8"
    ) as file:
        
        params = yaml.safe_load(file)

os.chdir(params['input_data_folderpath'])
load_dir = "load/"
summary_dir = "summary/"

In [4]:
# Run this one to load the actual workspace
root_directory = params['workspace_dir']
load_folderpath = root_directory + '/load'
aux_folderpath = root_directory + '/aux'
data_folderpath = root_directory + '/data'

In [ ]:
# Check set parameters
print(f"Working from {params['input_data_folderpath']}")
print(f"The pre-processed files will be placed in {root_directory}")

# 2. Check well-formed structure of measurements folder

In [ ]:
warnings, analysable_folderpaths = DataPreprocessor.check_structure_of_input_folder(
    params['input_data_folderpath'],
    subfolders_no=7,
    json_files_no_at_2nd_level=1,
    json_files_no_at_3rd_level=1,
    non_json_files_no_at_2nd_and_3rd_level=18
)

# Relate the analysable folderpaths to the working directory
analysable_folderpaths = [
    os.path.relpath(folderpath, params['input_data_folderpath']) 
    for folderpath in analysable_folderpaths
]

for warning in warnings:
    print('--> ' + warning)

# 3. Pre-process the data

In [ ]:
for folderpath in analysable_folderpaths:
    
    print('Preprocessing folder: ' + folderpath)
    print('-----------------------------------')

    if DataPreprocessor.hosts_gain_data(folderpath):
        json_default_values_filename = 'gainmeas_metadata.json'
    else:
        json_default_values_filename = 'darknoisemeas_metadata.json'

    path_to_json_default_values = os.path.join(
        folderpath, 
        json_default_values_filename
    )

    strips_ids = DataPreprocessor.grab_strip_IDs(
        path_to_json_default_values,
        3,
        verbose=params['verbose']
    )

    mypreprocessor = DataPreprocessor(
        folderpath,
        darknoise_base="W",
        gain_base='G',
        timestamp_prefix='ts_',
        verbose=params['verbose']
    )

    mypreprocessor.generate_meas_config_files(
        root_directory,
        load_folderpath,
        aux_folderpath,
        data_folderpath,
        wvf_skiprows_identifier='TIME,',
        path_to_json_default_values=path_to_json_default_values,
        sipms_per_strip=6,
        strips_ids=strips_ids,
        ask_for_inference_confirmation=False,
        verbose=params['verbose']
    )

    print('-----------------------------------')
    print('\n\n', end='')